## Match Index Notebook

In [2]:
import pandas as pd
import os

In [4]:
parent_dir = os.path.abspath('../..') # ".." means go back 1 directory!
print(parent_dir)

/Users/cjgimena/Desktop/Github/consulting-spring2025


In [29]:
file_path = os.path.join(parent_dir, 'data/mens/mens_results.csv')
file_path


'/Users/cjgimena/Desktop/Github/consulting-spring2025/data/mens/mens_results.csv'

In [30]:
matches = pd.read_csv(file_path)
matches.head()

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
0,"Dual Match: University of California, Los Ange...",2025-04-26,Emon Van Loben Sels,13.59,Alexander Bernard,13.33,"7-6(5), 5-7, 6-3"
1,"Dual Match: University of California, Los Ange...",2025-04-26,Alexander Hoogmartens,13.00,Jack Anthrop,13.66,"7-5, 0-6, 6-4"
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
3,"Dual Match: University of California, Los Ange...",2025-04-26,Preston Stearns,13.08,Aadarsh Tripathi,12.94,"6-3, 2-6, 6-2"
4,"Dual Match: University of California, Los Ange...",2025-04-26,Kaylan Bigun,13.17,William Jansen,13.24,"6-2, 7-5"


In [32]:
# Create a function that returns data in this format
file_path = os.path.join(parent_dir, 'data/index_sample.csv')
sample_output = pd.read_csv(file_path)
sample_output

,school_name,opponent,score,date
0,UCLA,Govind Nanda,7-6(5),04/25/25
1,Oregon,Vlad whatever,NaN,03/24/28
2,Univeristy of Washington,Cesar Bouchelaghem,6-4 7-6(3),03/24/28


In [ ]:
# Column Format Guidelines:

# school_name: Full School Name
# opponent: Full Name
# score: Respect to Player we are querying for (switch score if ucla player lost)
# date: 00/00/00